<a href="https://www.kaggle.com/code/kparthiban007/time-series-analysis?scriptVersionId=212304562" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="background-color:#FFEBCD; color:#FF8C00; font-size:25px; padding:10px; text-align:center; border-radius:25px;">
    Time Series Analysis
</div>

In [1]:
#Importing Necessary Libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# loading Data as a Pandas Dataframe
train_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")

train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [2]:
test_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [3]:
oil_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [4]:
holiday_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
holiday_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [5]:
holiday_df['type'].value_counts()

type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [6]:
holiday_df['locale'].value_counts()

locale
National    174
Local       152
Regional     24
Name: count, dtype: int64

In [7]:
transaction_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
transaction_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [8]:
store_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
store_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
store_df['city'].value_counts()

city
Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Manta             2
Latacunga         2
Machala           2
Ambato            2
Quevedo           1
Esmeraldas        1
Loja              1
Libertad          1
Playas            1
Daule             1
Babahoyo          1
Salinas           1
Puyo              1
Guaranda          1
Ibarra            1
Riobamba          1
Cayambe           1
El Carmen         1
Name: count, dtype: int64

In [10]:
print(f"Shape of the Data : {train_df.shape}")

Shape of the Data : (3000888, 6)


In [11]:
train_df.info() # information About the Data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [12]:
train_df.isnull().sum() # checking for null values

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [13]:
train_df.duplicated().sum() # checking for duplicate values

0

In [14]:
train_df['date'] =pd.to_datetime(train_df['date']) # converting object datatype into time datatype

In [15]:
categoric_columns = train_df.select_dtypes(include='object').columns
numeric_columns = train_df.select_dtypes(include=['int64','float64']).drop(columns ='id')
numeric_columns =numeric_columns.columns
categoric_columns ,numeric_columns

(Index(['family'], dtype='object'),
 Index(['store_nbr', 'sales', 'onpromotion'], dtype='object'))

In [16]:
train_df['sales'].value_counts()

sales
0.000       939130
1.000       115291
2.000        85959
3.000        68575
4.000        57846
             ...  
116.541          1
363.533          1
141.322          1
409.879          1
2419.729         1
Name: count, Length: 379610, dtype: int64

In [17]:
train_df['dataaset'] = "train"
test_df['dataset'] = "test"

df = pd.concat([train_df,test_df],axis = 0).reset_index(drop= True)
df.head()

,id,date,store_nbr,family,sales,onpromotion,dataaset,dataset
0,0,2013-01-01 00:00:00,1,AUTOMOTIVE,0.0,0,train,NaN
1,1,2013-01-01 00:00:00,1,BABY CARE,0.0,0,train,NaN
2,2,2013-01-01 00:00:00,1,BEAUTY,0.0,0,train,NaN
3,3,2013-01-01 00:00:00,1,BEVERAGES,0.0,0,train,NaN
4,4,2013-01-01 00:00:00,1,BOOKS,0.0,0,train,NaN


In [18]:
print(df.columns)
print(oil_df.columns)
print(holiday_df.columns)
print(store_df.columns)
print(transaction_df.columns)

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'dataaset',
       'dataset'],
      dtype='object')
Index(['date', 'dcoilwtico'], dtype='object')
Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')
Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')
Index(['date', 'store_nbr', 'transactions'], dtype='object')


In [19]:
holiday_df[holiday_df['transferred'] == True]


,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [20]:
holiday_df[holiday_df['type']=="Transfer"]

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


In [21]:
holiday_df['Holiday summary'] = (
    holiday_df['type'].astype(str) + " " +
    holiday_df['locale'].astype(str) + " " +
    holiday_df['locale_name'].astype(str) + " " 
)

In [22]:
hoilday_df = holiday_df.drop(columns =['type','locale','locale_name','description'],errors = 'ignore')
holiday_df['Holiday summary'].value_counts()


Holiday summary
Holiday National Ecuador                            60
Event National Ecuador                              56
Additional National Ecuador                         40
Holiday Local Riobamba                              12
Holiday Local Guaranda                              12
Holiday Local Latacunga                             12
Holiday Local Ambato                                12
Transfer National Ecuador                            8
Holiday Local Manta                                  6
Holiday Regional Cotopaxi                            6
Holiday Local Salinas                                6
Holiday Local Loja                                   6
Holiday Local Quito                                  6
Additional Local Quito                               6
Holiday Regional Santa Elena                         6
Holiday Regional Santo Domingo de los Tsachilas      6
Holiday Local Quevedo                                6
Holiday Local Ibarra                             

In [23]:
store_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [24]:
transaction_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [25]:
df = df.merge(oil_df,on='date',how='left')
df = df.merge(holiday_df ,on='date',how = 'left')
df = df.merge(store_df,on='store_nbr',how='left')
df = df.merge(transaction_df,on=['date', 'store_nbr'],how='left')

df.sample(10)

,id,date,store_nbr,family,sales,onpromotion,dataaset,dataset,dcoilwtico,type_x,locale,locale_name,description,transferred,Holiday summary,city,state,type_y,cluster,transactions
2716064,2716064,2017-03-09 00:00:00,17,SEAFOOD,12.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,C,12,NaN
731446,731446,2014-02-16 00:00:00,32,BABY CARE,0.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guayaquil,Guayas,C,3,NaN
617183,617183,2013-12-13 00:00:00,26,HOME APPLIANCES,0.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guayaquil,Guayas,D,10,NaN
357051,357051,2013-07-20 00:00:00,27,MEATS,268.733,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Daule,Guayas,D,1,NaN
2873500,2873500,2017-06-05 00:00:00,34,PERSONAL CARE,275.000,7,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guayaquil,Guayas,B,6,NaN
2275667,2275667,2016-07-04 00:00:00,10,LAWN AND GARDEN,0.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,C,15,NaN
290888,290888,2013-06-13 00:00:00,20,PET SUPPLIES,0.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN
2410360,2410360,2016-09-17 00:00:00,4,CLEANING,993.000,40,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,9,NaN
302817,302817,2013-06-19 00:00:00,6,DELI,210.969,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
605742,605742,2013-12-06 00:00:00,54,PLAYERS AND ELECTRONICS,0.000,0,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,El Carmen,Manabi,C,3,NaN


In [26]:
print("Summary of The Dataframe")

df.describe()

Summary of The Dataframe


,id,store_nbr,sales,onpromotion,dcoilwtico,cluster,transactions
count,3.029400e+06,3.029400e+06,3.000888e+06,3.029400e+06,21384.000000,3.029400e+06,0.0
mean,1.514700e+06,2.750000e+01,3.577757e+02,2.643830e+00,47.243333,8.481481e+00,NaN
std,8.745126e+05,1.558579e+01,1.101998e+03,1.233287e+01,0.753922,4.649735e+00,NaN
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,45.960000,1.000000e+00,NaN
25%,7.573498e+05,1.400000e+01,0.000000e+00,0.000000e+00,46.715000,4.000000e+00,NaN
50%,1.514700e+06,2.750000e+01,1.100000e+01,0.000000e+00,47.250000,8.500000e+00,NaN
75%,2.272049e+06,4.100000e+01,1.958473e+02,0.000000e+00,47.650000,1.300000e+01,NaN
max,3.029399e+06,5.400000e+01,1.247170e+05,7.410000e+02,48.590000,1.700000e+01,NaN


In [27]:
df["dcoilwtico"]=df["dcoilwtico"].fillna(method="ffill").fillna(method="bfill")

In [28]:
df['transactions'] = df.groupby('family')['transactions'].transform(
    lambda x: x.fillna(x.median())
)

In [29]:
df["is_holiday"]=df["Holiday summary"].fillna('Not Holiday').apply(lambda x: 0 if x == 'Not Holiday' else 1)

In [30]:
df.isnull().sum()


id                       0
date                     0
store_nbr                0
family                   0
sales                28512
onpromotion              0
dataaset             28512
dataset            3000888
dcoilwtico               0
type_x             3027618
locale             3027618
locale_name        3027618
description        3027618
transferred        3027618
Holiday summary    3027618
city                     0
state                    0
type_y                   0
cluster                  0
transactions       3029400
is_holiday               0
dtype: int64

In [31]:
df['transactions'] = df.groupby('family')['transactions'].transform(
    lambda x: x.fillna(x.median())
)

In [32]:
df['transactions'].fillna(method='ffill', inplace=True)
df['transactions'].fillna(df['transactions'].mean(), inplace=True)

In [33]:
df = df.drop(columns=['transferred', 'Holiday summary'], errors='ignore')


In [34]:
import datetime as dt


In [35]:
df['date'] = pd.to_datetime(df['date'])
df["year"]=df['date'].dt.year
df["month"]=df["date"].dt.month
df["day"]=df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek

In [36]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,dataaset,dataset,dcoilwtico,type_x,...,city,state,type_y,cluster,transactions,is_holiday,year,month,day,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,train,NaN,46.8,NaN,...,Quito,Pichincha,D,13,NaN,0,2013,1,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0,train,NaN,46.8,NaN,...,Quito,Pichincha,D,13,NaN,0,2013,1,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0,train,NaN,46.8,NaN,...,Quito,Pichincha,D,13,NaN,0,2013,1,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,train,NaN,46.8,NaN,...,Quito,Pichincha,D,13,NaN,0,2013,1,1,1
4,4,2013-01-01,1,BOOKS,0.0,0,train,NaN,46.8,NaN,...,Quito,Pichincha,D,13,NaN,0,2013,1,1,1


In [37]:
df['family'].value_counts()
df.groupby('family', as_index=False).agg(
    sales_count=('sales', 'count'),
    sales_mean=('sales', 'mean')
).sort_values(by='sales_mean', ascending=False)[:10]

,family,sales_count,sales_mean
12,GROCERY I,90936,3776.972100
3,BEVERAGES,90936,2385.793151
30,PRODUCE,90936,1349.352123
7,CLEANING,90936,1072.416744
8,DAIRY,90936,709.154889
5,BREAD/BAKERY,90936,463.336254
28,POULTRY,90936,350.532292
24,MEATS,90936,341.849965
25,PERSONAL CARE,90936,270.432513
9,DELI,90936,265.135067


In [38]:
df.groupby('store_nbr', as_index=False).agg(
    sales_count=('sales', 'count'),
    sales_mean=('sales', 'mean')
).sort_values(by='sales_mean', ascending=False)[:10]

,store_nbr,sales_count,sales_mean
43,44,55572,1117.245254
44,45,55572,980.673908
46,47,55572,916.798209
2,3,55572,908.405495
48,49,55572,781.330450
45,46,55572,753.905962
47,48,55572,646.604950
50,51,55572,592.231511
7,8,55572,548.734739
49,50,55572,515.601753


In [39]:
df=pd.get_dummies(df,columns=["store_nbr","family"],dtype=int, drop_first=True)

In [40]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder




# Function for Target Encoding multiple categorical columns
def target_encoding_multiple_columns(df, target_column, categorical_columns, dataset_column='dataset'):
    
    for col in categorical_columns:
        # Calculate target mean for each category in the column using training data
        target_mean = df[df[dataset_column] == 'train'].groupby(col)[target_column].mean()

        # Map the target mean encoding to the entire dataset
        df[f'{col}_encoded'] = df[col].map(target_mean)

        # Handle missing values for categories not present in training
        df[f'{col}_encoded'].fillna(df[target_column].mean(), inplace=True)

    return df

# Example usage: Apply Target Encoding to multiple columns
categorical_columns = ['month', 'day', 'year', 'day_of_week']
df = target_encoding_multiple_columns(df, target_column='sales', categorical_columns=categorical_columns)


# Drop unnecessary columns after encoding
columns_to_drop = [
    'locale_name', 'description', 'transferred', 'city', 'state', 'family',
    'month', 'day_of_week', 'day', 'year', 'cluster', 'type'
]
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
df.columns


Index(['id', 'date', 'sales', 'onpromotion', 'dataaset', 'dataset',
       'dcoilwtico', 'type_x', 'locale', 'type_y',
       ...
       'family_PLAYERS AND ELECTRONICS', 'family_POULTRY',
       'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD', 'month_encoded',
       'day_encoded', 'year_encoded', 'day_of_week_encoded'],
      dtype='object', length=101)

In [41]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 5: Split Data Based on Time Series Order

# Separate train and test datasets
train_df = df[df['dataset'] == 'train'].drop(columns=['dataset'], errors='ignore')
test_df = df[df['dataset'] == 'test'].drop(columns=['dataset'], errors='ignore')


# Drop unnecessary columns from both datasets
train_df = train_df.drop(columns=['transactions', 'id'], errors='ignore')
test_df = test_df.drop(columns=['transactions'], errors='ignore')

# Sort training data by date to preserve time series order
train_df = train_df.sort_values(by='date')

# Define the split point for time series validation
split_ratio = 0.99  # Adjust as needed
split_index = int(len(train_df) * split_ratio)

# Create training and validation sets
X_train = train_df.iloc[:split_index].drop(columns=['sales'])
y_train = train_df.iloc[:split_index]['sales']
X_val = train_df.iloc[split_index:].drop(columns=['sales'])
y_val = train_df.iloc[split_index:]['sales']

# Log transform the target variables to stabilize variance
y_train = np.log1p(y_train)  # Log transform training target
y_val = np.log1p(y_val)      # Log transform validation target

# Drop the 'date' column after feature extraction
X_train = X_train.drop(columns=['date'], errors='ignore')
X_val = X_val.drop(columns=['date'], errors='ignore')

X_test = test_df.drop(columns=['sales'], errors='ignore')